# 创建神经网络
这个文档以及接下来的几个文档，主要介绍的是如何使用这两个类来进行定义神经网络、初始化参数、以及保存和读取模型

In [1]:
from mxnet import nd
from mxnet.gluon import nn

net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Dense(units=256, activation='relu'),
        nn.Dense(units=10)
    )

print(net)


E:\APP\Anaconda3\envs\tensorflow1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dense(None -> 10, linear)
)


# 使用 nn.Block 来定义
事实上，nn.Sequential 是 nn.Block 的简单形式。首先来用 nn.Block 实现和上面同样的功能

In [2]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)
            
    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))


可以看到 nn.Block 的使用是通过创建一个它的自雷，其中至少包含了两个函数。
- __init__: 创建参数。上面的例子中我们使用了包含了参数的 dense 层
- forward(): 用来定义网络计算

并且创建的类的使用和之前的 net 也并没有什么不同

In [4]:
net2 = MLP()
print(net2)
net2.initialize()
x = nd.random_uniform(shape=(4,20))
y = net2(x)
y

MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)



[[ 0.01461641 -0.0089294  -0.07906897  0.07768989 -0.12873693 -0.01952744
  -0.13175528 -0.03132879  0.0363337  -0.08520765]
 [-0.05594679  0.00822184 -0.05789109  0.07252695 -0.11979484  0.00889583
  -0.19506356 -0.03687808  0.03777978 -0.11160932]
 [ 0.03809929  0.00328241 -0.0902581   0.06271226 -0.11470574 -0.0103777
  -0.07336813 -0.01885033  0.05428999 -0.07302649]
 [-0.00901174  0.0103955  -0.07959956  0.10375699 -0.13731928 -0.0023221
  -0.17693904 -0.06660189  0.06440312 -0.0798894 ]]
<NDArray 4x10 @cpu(0)>

下面仔细看一下 MLP 里面用的其他命令：
- super(MLP, self).__init__(**kwargs): 这句话调用的 nn.Block 的 __init__ 函数，它提供了 prefix（指定名字）和 params（指定模型参数）两个参数。
- self.name_scope(): 调用 nn.Block 提供的 name_scope 函数。作用是给里面的所有层和参数的名字上加上前缀（prefix),使得他们在系统里面是独一无二的，默认自动会生成前缀，我们也可以在创建的时候手动指定。推在在构建网络的时候，每个层至少在一个 name_scope 里面

In [5]:
print('default prefix:', net2.dense0.name)

net3 = MLP(prefix='another_mlp_')
print('customized prefixed:', net3.dense0.name)

default prefix: mlp1_dense0
customized prefixed: another_mlp_dense0


## nn.Block 到底是什么？
在 gluon 里面，nn.Block 是一个一般化的部件。整个神经网络可以是一个 nn.Block, 单个层也是一个 nn.Block。我们可以（近似）无限的嵌套 nn.Block 来构建新的 nn.Block

nn.Block 主要提供下面这些内容：
1. 存储参数
2. 描述 forward 如何执行
3. 自动求导